In [ ]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

# Imports

In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [2]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path

In [3]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [4]:
from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from scipy.signal import savgol_filter
import matplotlib.patches as patches

In [5]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run Functions/1.2/data_fct_Adrien_point_by_point

matplotlib.rcParams.update({'font.size': 25})

In [6]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [54]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Rivendel/20210113_sansbump/DC05_RF10/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 48, 58]
> slashcond | 5
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 2


In [55]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC05_RF10']
000 > DC05_RF10
> condition names ['DC', 'RF']
> number of points 1
> N_ions = 1024
> e_GMol = 50


In [56]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power  ,forcenumtry=28

data0 = data_name[0]
Tvar = PMandT[1]

> Points | 1
> Simulations pour chaque point | ['Try00', 'Try01']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Rivendel/20210113_sansbump/DC05_RF10/Try00/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
00 - 01  >  /home/adrian/RemoteFS/Rivendel/20210113_sansbump/DC05_RF10/Try01/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
Time elapsed:  2.205573000000001 s
Time elapsed:  0.036759550000000016 m


In [57]:
# Automatic retrieve of GMol energy
onlyfiles = [f for f in listdir(all_subdir[0]) if isfile(join(all_subdir[0], f)) and "SimuType" in f and ".dat" in f]
aux_energie = onlyfiles[1].find('0RFG')
aux_energie_unit = onlyfiles[1].find('eV')
E0_GMol = onlyfiles[1][aux_energie+4:aux_energie_unit+2]
E0_GMol = re.sub('k','000',E0_GMol)
print(E0_GMol)
E0_GMol = int(E0_GMol.strip('eV'))
print(E0_GMol)


50eV
50


In [58]:
E0_GMol = 50

# Plots
Nuage d'ions, Temp et Pos

## Load temperature from Langevin files

In [59]:
## Data loading with hardcode names
eta = 1.5e-20

plot_point = 0
plot_try = 1
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.15D-19'
dirname_hc = str(data0[plot_point][plot_try])+'/'
# Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

/home/adrian/RemoteFS/Rivendel/20210113_sansbump/DC05_RF10/Try01/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5


In [60]:
N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T =\
load_Temp_init_bin_Lan(dirname_hc+datainit_hc,1)

len save_T [55440]
[4] [4] [8] [8] [8] [55440]
N_ions 1024
j_save 990
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 3.16980250e-07 5.92120493e-04 4.89870552e-05
 4.63123404e+01 1.44205099e+00 6.77007414e+01]


In [61]:
print(dirname_hc,only_init[0].strip('.dat'))

/home/adrian/RemoteFS/Rivendel/20210113_sansbump/DC05_RF10/Try01/ Temp_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG


In [62]:
# Get T and fluo PM  All SimuType
onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and "SimuType" in f and ".dat" in f]
print(sort(onlyfiles))
only_init = [f for f in onlyfiles if "SimuTypeQ" in f]
ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(dirname_hc,only_init[0].strip('.dat'),flag_plot=1,fig_name='0',xlim1=(0,11),ylim1=(2e-4,500),ylim2=(2,65))
# tt2, T_CM2, T_aux2, PM2
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)
print(dirname_hc,only_init[0].strip('.dat')[4:])

['Temp_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG.dat'
 'xva_SimuType4_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.dat']
/home/adrian/RemoteFS/Rivendel/20210113_sansbump/DC05_RF10/Try01/ _SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG


In [82]:
def find_PM_variation_FinalT_absolute(file_dir2,file_name):
    i_aux = file_name.find('_N')+1
    file1 = 'SimuTypeQ_{}'.format(file_name[i_aux:].strip('50eV.dat'))    # for N=1024
    file3 = 'SimuType2_01_{}50eV'.format(file_name[i_aux:].strip('50eV.dat'))
    
    tt1, T_CM1, T_aux1, PM1 = load_T_and_PM_simu(file_dir2+'/Temp_'+file1)
    tt3, T_CM3, T_aux3, PM3 = load_T_and_PM_simu(file_dir2+'/Temp_'+file3)
    
    # 1ms is 2000 pts at 1pt/500ns
    
    aux = mean(PM1[-2000:])
    PM_variation = aux - mean(PM3[-2000:])
    T_variation  = ( mean(T_aux3[-2000:,0]) + mean(T_aux3[-2000:,1]) + mean(T_aux3[-2000:,2]) ) / 3
    
    SNR = np.abs( aux - mean(PM3[-2000:]) )/np.sqrt(aux) # Sig-to-Noi ratio considering Poisson noise
    
    return aux, mean(PM3[-2000:]), PM_variation, T_variation, SNR
a,b,c,d,e = find_PM_variation_FinalT_absolute(dirname_hc,only_init[0].strip('.dat'))
print(a*2,b*2,c*2,d,e)

72.82417000000001 12.658479999999999 60.16569000000001 334.8526203778965 4.985355492000767


In [63]:
# recover duration for each step of simu (i_free__fly, i_laser_fly ...)

address = all_subdir[plot_point]
print(address)
info_files = sort( [f for f in listdir(address) if isfile(join(address, f)) and ".info" in f] )
print(sort(info_files))


tmp_info = loadtxt(data0[plot_point][plot_try]+'/'+info_files[0],encoding ='utf-8',
            skiprows=1, usecols=(0),
            unpack=True)
eta = tmp_info[-2]
i_free__fly_Lan = tmp_info[-1]

tmp_info = loadtxt(data0[0][0]+'/'+'xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG.info',encoding ='utf-8',
            skiprows=0, usecols=(0),
            unpack=True)
i_laser_fly_quench = tmp_info[-2]
i_laser_fly = tmp_info[-1]

n_dt = 100
dt = 1/2e6

print(eta)
print(i_free__fly_Lan)
print(i_laser_fly_quench)
print(i_laser_fly)

/home/adrian/RemoteFS/Rivendel/20210113_sansbump/DC05_RF10/Try00
['Langevin_cooling.info'
 'xva_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuType4_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG.info']
1.5e-20
1000.0
8000.0
6000.0


In [25]:
# Plot temperature Langevin + md laser  ALL SimuType   ALL TEMP SEPARATED

tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
tt_md = ta[0]
T_aux = te[0]

tt = concatenate((tt_Lan,tt_md))
T_all = concatenate((save_T[:,4:],T_aux),axis=0)

PM = ti[0]

#fig_name = file_name[-9:]
figname = 'T_and_PM_plot_example'
fig1 = plt.figure(num=figname,clear='True')
fig1.set_size_inches(11.69,8.27)
# set height ratios for sublots
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
# the fisrt subplot
ax1 = plt.subplot(gs[0])

ax1.semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
ax1.semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
ax1.semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#semilogy(tt_Lan*1.e3,save_T[:,0], label='Tx',color='C0')
#semilogy(tt_Lan*1.e3,save_T[:,1], label='Ty',color='C1')
#semilogy(tt_Lan*1.e3,save_T[:,2], label='Tz',color='C2')
ax1.vlines(ta[1]*1e3,1e-4,1e4,color='red')
ax1.vlines(ta[2]*1e3,1e-4,1e4,color='red')
time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='green',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='orange',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
ax1.hlines(0.5,7.5,8.5, linestyle=':')
# ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
ylabel('T [K]')

# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')
,
# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

# ax2 = subplot(212,sharex=ax1)
ax2 = plt.subplot(gs[1], sharex = ax1)
plot(tt_md*1e3,PM, label='Tx')
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,70,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax2.vlines(time_switch*1e3,1e-4,70,color='green',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax2.vlines(time_switch*1e3,1e-4,70,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,90,color='purple')
ax2.grid()
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time [ms]')
ylabel('cts/500ns')

left, bottom, width, height = [0.25, 0.56, 0.39, 0.38] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

axins.semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
axins.semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
axins.semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
axins.hlines(0.5,7.4,7.66, linestyle=':')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
axins.vlines(time_switch*1e3,1e-4,1e4,color='orange',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly  +i_laser_fly_quench)*dt
axins.vlines(ta[2]*1e3,1e-4,1e4,color='red')

x1, x2, y1, y2 = 7.48, 7.65, 0.0002, 50 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.yaxis.set_label_position("right")
axins.yaxis.tick_right()


locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
axins.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
axins.yaxis.set_minor_locator(locmin)
axins.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

axins.grid(b=True, which='major', linestyle='-')
axins.grid(b=True, which='minor', linestyle='--')

# axins.tick_params(axis="y",direction="in", pad=-53)
mark_inset(ax1, axins, loc1=1, loc2=3, fc="none", ec="0.25")
# axins.invert_yaxis()
# plt.setp(axins.get_xticklabels(), visible=False)
# plt.setp(axins.get_yticklabels(), fontsize=10)

ax1.set_xlim(-0.15, 10.75)
ax1.set_ylim(0.0001, 1200)

plt.tight_layout()
subplots_adjust(hspace=0.0)
# savefig(fname = figname+'_'+re.sub('/','_',dirname_hc[31:-1])+'.png',dpi=300)
# savefig(fname+'_D'+str(Detuning)+'_50eV'+'.png',dpi=300)

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/figure.py:2267: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


In [64]:
final_tt = tt[-1]*1000
final_tt = float(str(final_tt)[0:5])
pts_to_1ms = round(len(tt)/final_tt) # 2 coups/µs * 1000 µs/ms
PM_start_offset = pts_to_1ms*tt[-len(PM)]*1000
index_ms = []
PM_1ms = []
tt_1ms_step = [2,3,4,5,6,7,8,9]
detec_eff = 1e-3
for k in tt_1ms_step:
    index_ms_tmp = int(floor(k/final_tt*len(tt)) ) - int(round(PM_start_offset))
    index_ms.append(index_ms_tmp)
    PM_1ms.append(sum( PM[index_ms_tmp-pts_to_1ms//2:index_ms_tmp+pts_to_1ms//2] )*detec_eff)

final_tt = tt[-1]*1000
final_tt = float(str(final_tt)[0:5])
pts_to_1us = 2 # 2 coups/µs
index_us = []
PM_1us = []
tt_1us_step = tt[1::2]+i_free__fly_Lan*dt
for k in range(1,2*len(tt_1us_step),2):
    PM_1us.append(sum(PM[k:k+2]))

In [65]:
gmol_address = os.path.join(all_subdir[0],'trj_SimuType4_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV')
tgmol, r_x,r_y,r_z, v_x,v_y,v_z, a_x,a_y,a_z = load_trj(gmol_address)
t_in = 0
t_out = 0
for j,k in enumerate(r_z):
    if k*1000 < -dim_nu[0,0,2]/2:
        t_in = tgmol[j]
    if k*1000 < dim_nu[0,0,2]/2:
        t_out = tgmol[j]
print(t_in*1000,t_out*1000)

7.513257062044534 7.515781430329185


In [80]:
# Plot temperature Langevin + md laser  ALL SimuType  ALL TEMP MEAN
matplotlib.rcParams.update({'font.size': 25})

tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
tt_md = ta[0]
T_aux = te[0]

tt = concatenate((tt_Lan,tt_md))
T_all = concatenate((save_T[:,4:],T_aux),axis=0)

PM = ti[0]

#fig_name = file_name[-9:]
figname = 'T_and_PM_plot_example'
fig1 = plt.figure(num=figname,clear='True')
fig1.set_size_inches(11.69,8.27)
# set height ratios for sublots
gs = gridspec.GridSpec(2, 1, height_ratios=[7, 3]) 
# the fisrt subplot
ax1 = plt.subplot(gs[0])

ax1.semilogy(tt*1e3,savgol_filter(mean(T_all,axis=1),25,1), label='Tx',color='red',lw=2.5) # savgol_filter(mean(T_all,axis=1),25,1)
#semilogy(tt_Lan*1.e3,save_T[:,0], label='Tx',color='C0')
#semilogy(tt_Lan*1.e3,save_T[:,1], label='Ty',color='C1')
#semilogy(tt_Lan*1.e3,save_T[:,2], label='Tz',color='C2')
# ax1.vlines(ta[1]*1e3,1e-4,1e4,color='red')
# ax1.vlines(ta[2]*1e3,1e-4,1e4,color='red')
time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-5,1e4,color='black')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-5,1e4,color='black',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
# ax1.vlines(time_switch*1e3,1e-4,1e4,color='black',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax1.vlines((ta[2])*1e3,1e-4,1e4,color='black')
# ax1.hlines(0.5,7.5,8.5, linestyle=':')
# ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
ylabel('T [K]')

ax1.annotate('1', xy=(0.22,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('2', xy=(2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('3', xy=((3.5+7.5)/2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
# ax1.annotate('4', xy=((7.5+7.57)/2,2.5e-4), xycoords='data',
# size=15, ha='center', va='top', color='xkcd:black',
# bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('5', xy=((7.5+10.5)/2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))

# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')

# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

# ax2 = subplot(212,sharex=ax1)
ax2 = plt.subplot(gs[1], sharex = ax1)

# ax2.plot(tt_md*1e3,PM, label='Tx')
ax2.scatter(tt_1ms_step,PM_1ms, label='cts/ms',marker='P',s=200,lw=1.5,color='black',zorder=100)
ax2.plot(tt_1us_step[:-500]*1e3,PM_1us[:-500], label='cts/µs',color='blue')
ax2.legend()
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,150,color='black')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
# ax2.vlines(time_switch*1e3,1e-4,150,color='black',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
# ax2.vlines(time_switch*1e3,1e-4,70,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,90,color='purple')
ax2.grid()
# ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=20))
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time [ms]')
ylabel('PM signal')

left, bottom, width, height = [0.345, 0.565, 0.35, 0.38] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

axins.semilogy(tt*1e3,mean(T_all,axis=1), label='Tx',color='red',lw=2.5)
# axins.hlines(0.5,7.4,7.66, linestyle=':')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
axins.vlines(time_switch*1e3,1e-4,1e4,color='black',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly  +i_laser_fly_quench)*dt
axins.vlines(ta[2]*1e3,1e-4,1e4,color='black')

axins.add_patch(
     patches.Rectangle(
        (t_in*1000, 3e-4),
        (t_out - t_in)*1000,
        0.1,
        edgecolor ='k',
        facecolor =  cc('grey',0.5),
        fill=True
     ) )

x1, x2, y1, y2 = 7.46, 7.59, 0.00012, 50 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

# axins.yaxis.tick_right()


locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
axins.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
axins.yaxis.set_minor_locator(locmin)
axins.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
plt.setp(axins.get_xticklabels(), backgroundcolor="w")
plt.setp(axins.get_yticklabels(), backgroundcolor="w")

axins.grid(b=True, which='major', linestyle='-')
axins.grid(b=True, which='minor', linestyle='--')

# axins.tick_params(axis="y",direction="in", pad=-53)
mark_inset(ax1, axins, loc1=1, loc2=3, fc="none", ec="0.25")

axins.annotate('3', xy=(7.48,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
axins.annotate('4', xy=(7.515,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
axins.annotate('5', xy=(7.55,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))

ax1.set_xlim(-0.15, 10.75)
ax1.set_ylim(0.00005, 1000)
ax2.set_ylim(-0, 135)

plt.tight_layout()
subplots_adjust(hspace=0.0,bottom=0.11,left=0.12,right=0.98,top=0.97)
savefig(fname = figname+'_'+re.sub('/','_',dirname_hc[31:-1])+'.png',dpi=300)
# savefig(figname+'_D'+str(Detuning)+'_50eV'+'.png',dpi=300)

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/figure.py:2267: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


In [ ]:
'20200903/DC05_RF10/Try28'
# tt*1e3,savgol_filter(mean(T_all,axis=1),25,1
# ax2.scatter(tt_1ms_step,PM_1ms, label='cts/ms',marker='P',s=200,lw=1.5,color='black',zorder=100)
# ax2.plot(tt_1us_step[:-500]*1e3,PM_1us[:-500], label='cts/µs',color='blue')
outfile = 'graph_example_20200903_DC05_RF10_Try28'
# savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)
np.savez(outfile, time=tt, temperature=T_all, time_1us=tt_1us_step, time_1ms=tt_1ms_step, PM_1us=PM_1us, PM_1ms=PM_1ms)

In [ ]:
# load check
outfile = 'graph_example_20200903_DC05_RF10_Try28'
with load(outfile+'.npz') as data:
    a = data['PM_1ms']
print(a)

In [74]:
savefig(figname+'.eps',dpi=600,format='eps')
savefig(figname+'.png',dpi=300)